# Aprendizagem de máquina
# Tarefa 4 - Programando Regressão Logística do Zero

Nessa tarefa é implementado o algoritmo de regressão logística para classes binárias. 

In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib
%matplotlib inline

# Algoritmo de regressão linear múltipla

### Função *score*

$\hat{P}(y=+1|\textbf{x},\hat{\textbf{w}}) = \displaystyle \frac{1}{1 + e^{- \hat{\textbf{w}} h(x)}}$

In [27]:
def prob_cond(W, X):
    return 1/(1 + np.exp(np.dot(-X, W)))

### Função para cálculo do passo gradiente

$\frac{\partial l(\textbf{w}^{(t)}}{\partial \textbf{w}_{j} } = \displaystyle \sum_{i=1}^{N}{ h_j(\textbf{x}_i)\left( y - \hat{P}(y=+1|\textbf{x}_i,\hat{\textbf{w}}^{(t)}) \right)}$

In [6]:
def step_gradient_vectorized(w_current,X,y,learningRate):
    
    w_gradient = np.zeros((5,1))
    
    pred = prob_cond(w_current, X)
    
    tam = pred.shape[0]
    pred = pred.reshape(tam,)

    for i in range(0,len(X.T)):
        
        X2 = X[:,i]
        
        w_gradient[i] = np.sum(np.multiply(X2, (y - pred ) ) )
        
        w_current[i] = w_current[i] + (learningRate * w_gradient[i])

    new_w = w_current
    
    return [new_w, w_gradient]

In [33]:
def gradient_ascent_runner_vectorized(starting_w, X, y, learning_rate, epsilon):
    w = starting_w
    
    grad = np.array([np.inf,np.inf,np.inf,np.inf,np.inf]) 
    
    X = np.array(X)
    y = np.array(y)
    
    while (np.linalg.norm(grad)>epsilon):

        w,w_gradient = step_gradient_vectorized(w, X, y, learning_rate)
        
        grad = np.array(w_gradient)
        
    return w

### Função de predição de classe

In [8]:
def predict(X,W,grau_certeza): #Recebe o conjunto X de features e o vetor W de coeficientes
    
    X = np.array(X)
    
    y = np.zeros(X.shape[0])
    
    for i in range(0,len(X)):

        if  prob_cond(W,X[i]) > grau_certeza:
            y[i] = 1
        else:
            y[i] = 0
    
    return y

## Configuração do ambiente e execução do algoritmo de regressão logísitca
### Leitura dos dados

In [9]:
iris = pd.read_csv("iris.data", names = ["sepal length", "sepal width", "petal length", "petal width", "class"])
iris = iris.replace(['Iris-versicolor', 'Iris-virginica'], 'Iris-no-setosa')

ones = np.ones(len(iris))
ones = pd.DataFrame(ones)
iris.insert(0, '1s', ones)

iris.head()

,1s,sepal length,sepal width,petal length,petal width,class
0,1.0,5.1,3.5,1.4,0.2,Iris-setosa
1,1.0,4.9,3.0,1.4,0.2,Iris-setosa
2,1.0,4.7,3.2,1.3,0.2,Iris-setosa
3,1.0,4.6,3.1,1.5,0.2,Iris-setosa
4,1.0,5.0,3.6,1.4,0.2,Iris-setosa


### Separação do conjunto das variáveis de entrada do conjunto da variável alvo

In [10]:
X = iris.loc[:, ["1s", "sepal length", "sepal width", "petal length", "petal width"]]
y = iris.loc[:, ["class"]]

y = y.replace(['Iris-setosa'], 1)
y = y.replace(['Iris-no-setosa'], 0)

y['class'] = pd.to_numeric(y['class'])

y = np.array(y)

y = y.reshape(y.shape[0],)

## Execução

Aplicação do algoritmo de aprendizagem ao conjunto de treinamento definido acima:

In [34]:
init_w = np.zeros((5,1))

# learning_rate = 0.05
learning_rate = 0.00009
# learning_rate = 0.0000001
epsilon = 0.05

print("Iniciando gradiente ascendente w0 = {0}, w1 = {1}, w2 = {2}, w3 = {3}, w4 = {4}".format(init_w[0], init_w[1], init_w[2], init_w[3], init_w[4]))
print("Running...")

w = gradient_ascent_runner_vectorized(init_w, X, y, learning_rate, epsilon)
print("Gradiente ascendente convergiu com w0 = {0}, w1 = {1}, w2 = {2}, w3 = {3}, w4 = {4}".format(w[0], w[1], w[2], w[3], w[4]))

Iniciando gradiente ascendente w0 = [ 0.], w1 = [ 0.], w2 = [ 0.], w3 = [ 0.], w4 = [ 0.]
Running...
Gradiente ascendente convergiu com w0 = [ 0.59295669], w1 = [ 0.94624963], w2 = [ 3.27518092], w3 = [-5.12522917], w4 = [-2.40615172]


# Comparação dos coeficientes do algoritmo acima com aqueles do Scikit-learn

Vejamos se os coeficientes gerados pela função de regressão logística do pacote de machine learning **scikit-learn** são semelhantes àqueles encontrados pelo algoritmo aqui programado:

Considerando que foi criada uma coluna com 1s para calcular o w0 no algoritmo acima, e que o scikit não necessita dela, iremos desconsiderá-la a seguir:

In [16]:
X_new = iris.loc[:, ["sepal length", "sepal width", "petal length", "petal width"]]

No **sckit** há o regressor logístico SGDClassifier, que opera de forma semelhante ao algoritmo implementado acima quando utilizando a função de perda *log likelihood*.

In [36]:
from sklearn.linear_model import SGDClassifier

model_lreg = SGDClassifier(tol=epsilon, eta0=learning_rate, learning_rate='constant', loss='log', penalty='none').fit(X_new, y)
print("Coeficientes: w0 = {0}, w1 = {1}, w2 = {2}, w3 = {3}, w4 = {4}".format(model_lreg.intercept_[0], model_lreg.coef_[0,0], model_lreg.coef_[0,1], model_lreg.coef_[0,2], model_lreg.coef_[0,3]))

Coeficientes: w0 = -0.003344782431728037, w1 = -0.026860387303040415, w2 = -0.006971360846545073, w3 = -0.032722560635187824, w4 = -0.01238640317713921


Apesar de utilizando os mesmos *epslon* e *learning_rate*, os valores dos coeficientes não apresentaram-se semelhantes entre as duas soluções utilizadas.